# Mandatory Assignment 2

#### Part III: LLM


***
Here you should deploy an LLM as classifier for the same dataset, banking77, that you used in Parts 1 (Bag of Words) and 2 (BERT). You should to define a prompt that gives the LLM information about the classes. You should report results using a classification report. Also, include some reflections on your results: how do they compare with the results from Parts I, and II? You should also reflect on the prompt you designed, and explore variations on that prompt.



***